In [ ]:
def points_from_dist_angle(dist, angle):
    num = len(dist)
    points = np.zeros((num+1,2))
    ang = 0
    for i in range(num):
        ang = ang + angle[i]
        points[i+1,0] = points[i,0] - dist[i]*np.sin(ang*np.pi/180)
        points[i+1,1] = points[i,1] + dist[i]*np.cos(ang*np.pi/180)
    return points

def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

def rotate_from_video(video_file):
    cap = cv2.VideoCapture(video_file)
    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.05,
                       minDistance = 7,
                       blockSize = 7 )
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    # list of distances between frames
    distances = []
    while(1):
        for j in range(3):
            ret,frame = cap.read()
        if ret == False:
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        if p1 is None:
            print('   -  no feature point detected, process break.')
            break
        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]
        # calcule the move distance
        distance = []
        for i in range(len(good_new)):
            distance.append(good_new[i][0]-good_old[i][0])
        distance = reject_outliers(np.asarray(distance))
        distance = np.mean(distance)
        distances.append(distance)
        # update old frame
        old_gray = frame_gray.copy()
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    cap.release()
    #print(distances)
    rotates = np.asarray(distances)[0:-1]
    return np.sum(rotates)*54/video_width

def rotate_and_translation_from_video(video_file):
    cap = cv2.VideoCapture(video_file)
    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.05,
                       minDistance = 7,
                       blockSize = 7 )
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    # list of distances between frames
    rotates = []
    ratio_translate = []
    while(1):
        for j in range(3):
            ret,frame = cap.read()
        if ret == False:
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        if p1 is None:
            print('   -  no feature point detected, process break.')
            break
        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]
        # calcule the move distance
        rotate = []
        ratio = []
        for i in range(len(good_new)):
            theta = np.arctan2(abs(good_old[i][1]-video_height/2), (good_old[i][0]-video_width/2))
            dx = (good_new[i][1]-good_old[i][1])/np.sin(theta)
            radio = abs(dx)/(np.sqrt((good_old[i][1]-video_height/2)**2
                           + (good_old[i][0]-video_width/2)**2))
            rotate.append((good_new[i][0]-good_old[i][0]) -
                          abs(good_new[i][1]-good_old[i][1])*np.tan(theta))
            ratio.append(radio)
        ratio = reject_outliers(np.asarray(ratio))
        ratio = np.mean(ratio)
        ratio_translate.append(ratio)

        rotate = reject_outliers(np.asarray(rotate))
        rotate = np.mean(rotate)
        rotates.append(rotate)
        # update old frame
        old_gray = frame_gray.copy()
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    cap.release()
    rotates = np.asarray(rotates)[1:-1]
    ratio_translate = np.nan_to_num(np.asarray(ratio_translate)[1:-1])
    return np.sum(ratio_translate)*30, np.sum(rotates)*54/video_width
